In [1]:
!activate base

In [1]:
    from paddleocr import PaddleOCR,draw_ocr
    import PIL
    from PIL import ImageFont
    from PIL import Image
    from PIL import ImageTk
    import PIL.Image
    from imutils import paths
    import cv2
    from matplotlib import pyplot as plt
    %matplotlib inline
    import os
    import shutil
    import pandas as pd
    from pylab import imshow
    import numpy as np
    import torch
    import seaborn as sns
    import albumentations as albu
    from albumentations.pytorch.transforms import ToTensorV2
    from iglovikov_helper_functions.dl.pytorch.utils import tensor_from_rgb_image
    from iglovikov_helper_functions.utils.image_utils import load_rgb
    from datasouls_antispoof.pre_trained_models import create_model
    from datasouls_antispoof.class_mapping import class_mapping
    from imutils.video import VideoStream
    import tensorflow as tf
    import argparse
    import imutils
    import pickle
    import time 
    import tensorflow_hub as hub

In [2]:
def display(im_path):
    dpi = 80
    im_data = plt.imread(im_path)    
    height, width  = im_data.shape[:2]
    
    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)    
    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])    
    # Hide spines, ticks, etc.
    ax.axis('off')    
    # Display the image.
    ax.imshow(im_data, cmap='gray')    
    plt.show()


def paddle_man(img_path):
    ocr = PaddleOCR(det_model_dir='paddleOCR_V2_EN_GEN_SRV_MOB_T/ch_PP-OCRv2_det_distill_train', rec_model_dir='paddleOCR_V2_EN_GEN_SRV_MOB_T/ret_model/ch_PP-OCRv2_rec_train',cls_model_dir='paddleOCR_V2_EN_GEN_SRV_MOB_T/cls_model/ch_ppocr_mobile_v2.0_cls_train',use_angle_cls=True) ;
    result = ocr.ocr(img_path,cls=True);
    txts = [line[1][0] for line in result]
    scores = [line[1][1] for line in result]
    
    # draw result
    image = Image.open(img_path).convert('RGB')
    boxes = [line[0] for line in result]
    txts = [line[1][0] for line in result]
    scores = [line[1][1] for line in result]
    font = ImageFont.load_default()
    
    im_show = draw_ocr(image, boxes, txts, scores, font_path='L:/CV/PaddleOCR/PaddleOCR/doc/fonts/simfang.ttf')
    im_show = Image.fromarray(im_show)
    im_show.save('result.jpg')
    im = Image.open("result.jpg")
    im.show() 
    return txts,scores

In [3]:
#https://stackoverflow.com/questions/28816046/
#displaying-different-images-with-actual-size-in-matplotlib-subplot
%matplotlib inline
def display(im_path):
    dpi = 80
    im_data = plt.imread(im_path)    
    height, width  = im_data.shape[:2]
    
    # What size does the figure need to be in inches to fit the image?
    figsize = width / float(dpi), height / float(dpi)    
    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])    
    # Hide spines, ticks, etc.
    ax.axis('off')    
    # Display the image.
    ax.imshow(im_data, cmap='gray')    
    plt.show()

In [4]:
def main_code(img):
    #Remove previous results
    isdir = os.path.isdir('runs/detect')
    if isdir == True:
        shutil.rmtree(r'runs/detect')
    else:
        pass
    
    #Convert image to gray scale
    
    read_img = cv2.imread(img)
    gray_image=cv2.cvtColor(read_img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite('temp/gray_image.jpg', gray_image)
    
    #Detect VIN Region
    !python yolov5/detect.py --weights yolov5/yolov5/weights/large/best.pt --img 416 --conf 0.6 --source temp/gray_image.jpg --project runs/detect/ --name exp --save-crop
    #import torch
    #model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/yolov5/weights/best.pt')  # default
    #imgs = ['temp/gray_image.jpg'] 
    #results = model(imgs)
    #results.xyxy[0]  
    #results.pandas().xyxy[0]  
    #crops = results.crop(save=True)
    
    
    ##Check if directry present(True if object detection is correct)
    detect_folder='runs/detect/exp/crops/Letters/'
    isdir = os.path.isdir(detect_folder)
    if isdir == False:
        txts,score=paddle_man(img)   
    else:
        print('Dir exist')
        file = os.listdir(detect_folder)
        file=file[0]
        path = (detect_folder+""+file)
        src = cv2.imread(path)
        image = cv2.rotate(src, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
        cv2.imwrite("runs/temp/"+file, image)    
        #Convert to inverted imge
        gray_image_file='runs/temp/'+ file
        image_read=cv2.imread(gray_image_file)
        inverted_image = cv2.bitwise_not(image_read)
        cv2.imwrite("runs/temp/inverted.jpg", inverted_image)
        #Pass images to OCR
        txts,score=paddle_man("runs/temp/inverted.jpg")
        
    #torch.cuda.empty_cache()
    
    return txts,score

In [5]:
def liveness_det_one(img_in):
    ##Livesness detection
    ##Datasouls
    ###   
    #model = create_model("tf_efficientnet_b3_ns")
    model1 = create_model("tf_efficientnet_b3_ns")
    model2 = create_model("swsl_resnext50_32x4d")
    model1.eval();
    model2.eval();
    image_replay = load_rgb(img)
    #imshow(image_replay)
    ###
    transform = albu.Compose([albu.PadIfNeeded(min_height=400, min_width=400),
                              albu.CenterCrop(height=400, width=400), 
                              albu.Normalize(p=1), 
                              albu.pytorch.ToTensorV2(p=1)], p=1)
    
    with torch.no_grad():
        prediction1 = model1(torch.unsqueeze(transform(image=image_replay)['image'], 0)).numpy()[0]
        prediction2 = model2(torch.unsqueeze(transform(image=image_replay)['image'], 0)).numpy()[0]

    df1 = pd.DataFrame({"prediction": prediction1, "class_name": class_mapping.keys()})
    df2 = pd.DataFrame({"prediction": prediction2, "class_name": class_mapping.keys()})
    #sns.barplot(data=df1, x="prediction", y="class_name") 
    #sns.barplot(data=df2, x="prediction", y="class_name") 
    
    res1_df=df1.loc[df1['prediction'].idxmax()]
    res2_df=df2.loc[df2['prediction'].idxmax()]
    redf1,redf2,pred1,pred2=res1_df.class_name,res2_df.class_name,res1_df.prediction,res2_df.prediction
    ###
    return redf1,redf2,pred1,pred2
    
    
    
    ##Livesness detection
    ##Neural Network(Binary classification)

def liveness_det_two(img):
    liveness_model = tf.keras.models.load_model('LIVENESS_DETECT/face_liveness_detection/model',custom_objects={'KerasLayer':hub.KerasLayer})
    le = pickle.loads(open('LIVENESS_DETECT/face_liveness_detection/label_encoder','rb').read())
    
    ##Read image and resize --
    frame = cv2.imread(img)
    face = cv2.resize(frame, (224,224))
    
    face = face.astype('float') / 255.0 
    face = tf.keras.preprocessing.image.img_to_array(face)
    # tf model require batch of data to feed in
    # so if we need only one image at a time, we have to add one more dimension
    # in this case it's the same with [face]
    face = np.expand_dims(face, axis=0)
    
    preds = liveness_model.predict(face)[0]
    j = np.argmax(preds)
    label = le.classes_[j]
    result2= 'fake' if label ==1 else 'real'
    res3df=pd.DataFrame(preds,columns=['dd'])
    tmp_df=res3df.loc[res3df['dd'].idxmax()]
    pred3=tmp_df.dd
    return result2,pred3


In [10]:
##Testing##
torch.cuda.empty_cache()
#folder='test/HERO_PRINTED/replay/'       ##Images captired from computer screen
#folder='test/HERO_PRINTED/bw_p/'         ##Images captured from printed paper (B/W)
#folder='test/HERO_PRINTED/color_p/'      ##Images captured from color printed paper
folder='test/HERO_PRINTED/real/'          ##Real captured images

file='real10.jpeg'


img=(folder+""+file)
liv3,pred3=liveness_det_two(img);
##Model3
if liv3 == 'fake':
    print("Spoof detection result:",liv3," --> Accuracy:",pred3)
    #!python pp.py
else:
#display(img)
    txts,score=main_code(img);
    #redf1,redf2,pred1,pred2=liveness_det_one(img);
    #liv3,pred3=liveness_det_two(img);


    ###Printing Results
    print("\n")
    print("Recognised Text :",txts)
    print("Accuracy :",score)
    print()
    ###Liveness Detection Results###
    ##Model1 & Model2
    #print("Image Liveness Detection Result:\n")
    #print("Model1 :",redf1," --> Accuracy:",pred1)
    #print("Model2 :",redf2," --> Accuracy:",pred2)
    print("Fake image detection result:",liv3," --> Accuracy:",pred3)
    




Dir exist
[2022/03/03 23:00:26] root WARNING: version PP-OCRv2 not support cls models, auto switch to version PP-OCR


detect: weights=['yolov5/yolov5/weights/large/best.pt'], source=temp/gray_image.jpg, imgsz=[416, 416], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect/, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.0-131-gc77a5a8 torch 1.10.2+cu113 CUDA:0 (GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model Summary: 367 layers, 46108278 parameters, 0 gradients, 107.8 GFLOPs
image 1/1 L:\CV\MAIN_PRO_POC\temp\gray_image.jpg: 416x256 1 Letters, Done. (0.020s)
Speed: 1.0ms pre-process, 20.0ms inference, 4.4ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs\detect\exp


Namespace(benchmark=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='paddleOCR_V2_EN_GEN_SRV_MOB_T/cls_model/ch_ppocr_mobile_v2.0_cls_train', cls_thresh=0.9, cpu_threads=10, det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='paddleOCR_V2_EN_GEN_SRV_MOB_T/ch_PP-OCRv2_det_distill_train', det_pse_box_thresh=0.85, det_pse_box_type='box', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_polygon=True, e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set='totaltext', enable_mkldnn=False, gpu_mem=500, hel

In [51]:
####OPENCV


import cv2 

key = cv2. waitKey(1)
webcam = cv2.VideoCapture(0)
while True:
    try:
        check, frame = webcam.read()
        cv2.imshow("Capturing", frame)
        key = cv2.waitKey(1)
        if key == ord('s'): 
            cv2.imwrite(filename='saved_img.jpg', img=frame)
            webcam.release()
            img_new = cv2.imread('saved_img.jpg', cv2.IMREAD_GRAYSCALE)
            img_new = cv2.imshow("Captured Image", img_new)
            cv2.waitKey(1650)
            cv2.destroyAllWindows()

            img_ = cv2.imread('saved_img.jpg', cv2.IMREAD_ANYCOLOR)
        
            break
        elif key == ord('q'):
            print("Turning off camera.")
            webcam.release()
            print("Camera off.")
            print("Program ended.")
            cv2.destroyAllWindows()
            break
        
    except(KeyboardInterrupt):
        print("Turning off camera.")
        webcam.release()
        print("Camera off.")
        print("Program ended.")
        cv2.destroyAllWindows()
        break
    

img=('saved_img.jpg')
liv3,pred3=liveness_det_two(img);
##Model3
if liv3 == 'fake':
    print("Spoof detection result:",liv3," --> Accuracy:",pred3)
else:
#display(img)
    txts,score=main_code(img);
    #redf1,redf2,pred1,pred2=liveness_det_one(img);
    #liv3,pred3=liveness_det_two(img);


    ###Printing Results
    print("\n")
    print("Recognised Text :",txts)
    print("Accuracy :",score)
    print()
    ###Liveness Detection Results###
    ##Model1 & Model2
    #print("Image Liveness Detection Result:\n")
    #print("Model1 :",redf1," --> Accuracy:",pred1)
    #print("Model2 :",redf2," --> Accuracy:",pred2)
    print("Fake image detection result:",liv3," --> Accuracy:",pred3)

Spoof detection result: fake  --> Accuracy: 0.99850625


In [ ]:

image = cv2.imread('result - Copy.jpg',cv2.IMREAD_UNCHANGED)
position = (10,50)
cv2.putText(image,"Python Examples",position,cv2.FONT_HERSHEY_SIMPLEX,1,(0, 255, 0),3) 
cv2.imwrite('result - Copy.jpg', image)

In [7]:
################TKINTER
import numpy as np
import cv2

def fh():
    img=('cap_img.jpg')
    fake_res='this is a fake image'
    real_res='this is a real image'
    
    
    liv3,pred3=liveness_det_two(img);
    ##Model3
    if liv3 == 'fake':
        print("Spoof detection result:",liv3," --> Accuracy:",pred3)
    
        image = cv2.imread(img,cv2.IMREAD_UNCHANGED)
        position = (10,50)
        cv2.putText(image,"this is a fake image",position,cv2.FONT_HERSHEY_SIMPLEX,1,(0, 0, 255),3) 
        cv2.imwrite(img, image)
        !python pp.py
    
    else:
    
        txts,score=main_code(img);
    
        print("Spoof detection result:",liv3," --> Accuracy:",pred3)
        image = cv2.imread(img,cv2.IMREAD_UNCHANGED)
        position = (10,50)
        cv2.putText(image,"this is a real image",position,cv2.FONT_HERSHEY_SIMPLEX,1,(0, 255, 0),3) 
        cv2.imwrite(img, image)
        !python pp.py
    
    
        ###Printing OCR Results
        print("\n")
        print("Recognised Text :",txts)
        print("Accuracy :",score)
        print()
    
    
    
    

In [8]:
################TKINTER


import tkinter as tk
import cv2
import time
import datetime as dt
import argparse
import matplotlib.pyplot as plt

from tkinter import *  


class App:
    def __init__(self, window, window_title, video_source=0):
        self.window = window
        self.window.title(window_title)
        self.video_source = video_source
        self.ok=False

        #timer
        self.timer=ElapsedTimeClock(self.window)

        # open video source (by default this will try to open the computer webcam)
        self.vid = VideoCapture(self.video_source)

        # Create a canvas that can fit the above video source size
        self.canvas = tk.Canvas(window, width = self.vid.width, height = self.vid.height)
        self.canvas.pack()

        # Button that lets the user take a snapshot
        self.btn_snapshot=tk.Button(window, text="Snapshot", command=self.snapshot)
        self.btn_snapshot.pack(side=tk.LEFT)

        #video control buttons

        self.btn_start=tk.Button(window, text='START', command=self.open_camera)
        self.btn_start.pack(side=tk.LEFT)

        self.btn_stop=tk.Button(window, text='STOP', command=self.close_camera)
        self.btn_stop.pack(side=tk.LEFT)

        # quit button
        self.btn_quit=tk.Button(window, text='QUIT', command=window.destroy)
        self.btn_quit.pack(side=tk.LEFT)

        # After it is called once, the update method will be automatically called every delay milliseconds
        self.delay=10
        self.update()

        self.window.mainloop()

    def snapshot(self):
        # Get a frame from the video source
        ret,frame=self.vid.get_frame()

        if ret:
            cv2.imwrite("cap_img"+".jpg",cv2.cvtColor(frame,cv2.COLOR_RGB2BGR))
            fh()




    def open_camera(self):
        self.ok = True
        self.timer.start()
        print("camera opened => Recording")



    def close_camera(self):
        self.ok = False
        self.timer.stop()
        print("camera closed => Not Recording")

       
    def update(self):

        # Get a frame from the video source
        ret, frame = self.vid.get_frame()
        if self.ok:
            self.vid.out.write(cv2.cvtColor(frame,cv2.COLOR_RGB2BGR))

        if ret:
            self.photo = ImageTk.PhotoImage(image = PIL.Image.fromarray(frame))
            self.canvas.create_image(0, 0, image = self.photo, anchor = tk.NW)
        self.window.after(self.delay,self.update)


class VideoCapture:
    def __init__(self, video_source=0):
        # Open the video source
        self.vid = cv2.VideoCapture(video_source)
        if not self.vid.isOpened():
            raise ValueError("Unable to open video source", video_source)

        # Command Line Parser
        #args=CommandLineParser().args

        
        #create videowriter

        # 1. Video Type
        VIDEO_TYPE = {
            'avi': cv2.VideoWriter_fourcc(*'XVID'),
            #'mp4': cv2.VideoWriter_fourcc(*'H264'),
            'mp4': cv2.VideoWriter_fourcc(*'XVID'),
        }

        self.fourcc=VIDEO_TYPE['mp4']

        # 2. Video Dimension
        STD_DIMENSIONS =  {
            '480p': (640, 480),
            '720p': (1280, 720),
            '1080p': (1920, 1080),
            '4k': (3840, 2160),
        }
        res=STD_DIMENSIONS['480p']
        #print(args.name,self.fourcc,res)
        self.out = cv2.VideoWriter('output'+'.'+'output',self.fourcc,10,res)

        #set video sourec width and height
        self.vid.set(3,res[0])
        self.vid.set(4,res[1])

        # Get video source width and height
        self.width,self.height=res


    # To get frames
    def get_frame(self):
        if self.vid.isOpened():
            ret, frame = self.vid.read()
            if ret:
                # Return a boolean success flag and the current frame converted to BGR
                return (ret, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
            else:
                return (ret, None)
        else:
            return (ret, None)

    # Release the video source when the object is destroyed
    def __del__(self):
        if self.vid.isOpened():
            self.vid.release()
            self.out.release()
            cv2.destroyAllWindows()


class ElapsedTimeClock:
    def __init__(self,window):
        self.T=tk.Label(window,text='00:00:00',font=('times', 20, 'bold'), bg='green')
        self.T.pack(fill=tk.BOTH, expand=1)
        self.elapsedTime=dt.datetime(1,1,1)
        self.running=0
        self.lastTime=''
        t = time.localtime()
        self.zeroTime = dt.timedelta(hours=t[3], minutes=t[4], seconds=t[5])
        # self.tick()

 
    def tick(self):
        # get the current local time from the PC
        self.now = dt.datetime(1, 1, 1).now()
        self.elapsedTime = self.now - self.zeroTime
        self.time2 = self.elapsedTime.strftime('%H:%M:%S')
        # if time string has changed, update it
        if self.time2 != self.lastTime:
            self.lastTime = self.time2
            self.T.config(text=self.time2)
        # calls itself every 200 milliseconds
        # to update the time display as needed
        # could use >200 ms, but display gets jerky
        self.updwin=self.T.after(100, self.tick)


    def start(self):
            if not self.running:
                self.zeroTime=dt.datetime(1, 1, 1).now()-self.elapsedTime
                self.tick()
                self.running=1

    def stop(self):
            if self.running:
                self.T.after_cancel(self.updwin)
                self.elapsedTime=dt.datetime(1, 1, 1).now()-self.zeroTime
                self.time2=self.elapsedTime
                self.running=0


def main():
    # Create a window and pass it to the Application object
    App(tk.Tk(),'Video Recorder')

main()

Exception in Tkinter callback
Traceback (most recent call last):
  File "L:\ANACONDA\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\rajes\AppData\Local\Temp/ipykernel_8620/707825685.py", line 59, in snapshot
    fh()
  File "C:\Users\rajes\AppData\Local\Temp/ipykernel_8620/4077823848.py", line 11, in fh
    liv3,pred3=liveness_det_two(img);
  File "C:\Users\rajes\AppData\Local\Temp/ipykernel_8620/224066039.py", line 43, in liveness_det_two
    frame = cv2.imread(img)
NameError: name 'img' is not defined


'cuda:0'

In [73]:
##Liveness DetectOOnly###
##Liveness DetectOOnly###

#Variables
result=[]
img=[]
imagePaths=[]
accuracy=[]
imagePaths = sorted(list(paths.list_images('test/HERO_PRINTED/replay/')))
for img in imagePaths:

    liv3,pred3=liveness_det_two(img);
    result.append(liv3)
    accuracy.append(pred3)

    ##Model3
#print("result of all images:",result)
#print("image count:",len(result))
data=pd.DataFrame(result,accuracy)
data.to_csv('replay.csv')